In [1]:
import pandas as pd
from classifier.classifier import Classifier
from classifier.classifier_factory import ClassifierFactory
from database.market import Market
from database.sec import SEC
from database.adatabase import ADatabase
from processor.processor import Processor as p
from datetime import datetime, timedelta
from tqdm import tqdm
import numpy as np
from modeler_strats.universal_modeler import UniversalModeler

In [2]:
## initializing classifier_class
classifier_class = ClassifierFactory.build(Classifier.WEEKLY_CRYPTO_FASTSLOW_CLASSIFIER)

In [3]:
backtest_start_date = datetime(2016,1,1)
backtest_end_date = datetime(2023,1,1)
current_year = datetime.now().year

In [4]:
## creating database objects
market = Market()
modeler_strat = UniversalModeler()

In [5]:
market.connect()
training_sets = []
for ticker in tqdm(["BTC"]):
    try:
        prices = market.retrieve_ticker_prices(classifier_class.asset_class.value,ticker)
        prices = p.column_date_processing(prices)
        ticker_data = classifier_class.training_set(ticker,prices)
        training_sets.append(ticker_data)
    except Exception as e:
        print(str(e))
        continue
market.disconnect()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 21.27it/s]


In [6]:
data = pd.concat(training_sets)

In [7]:
training_data = data.dropna().copy().sort_values(["year","week"])

In [8]:
training_data = training_data.groupby(["year","quarter","week","ticker"]).mean().reset_index()

In [9]:
training_data

,year,quarter,week,ticker,classification_prediction
0,2011,4,52,BTC,0.000000
1,2012,1,1,BTC,0.428571
2,2012,1,2,BTC,1.000000
3,2012,1,3,BTC,1.000000
4,2012,1,4,BTC,1.000000
...,...,...,...,...,...
634,2023,2,21,BTC,1.000000
635,2023,2,22,BTC,1.000000
636,2023,2,23,BTC,0.285714
637,2023,2,24,BTC,0.000000


In [10]:
classifier_class.db.connect()
relevant_columns = list(set(list(training_data.columns)) - set(classifier_class.factors))
classifier_class.db.store("sim",training_data[relevant_columns])
classifier_class.db.disconnect()

In [11]:
# ranker_class.db.connect()
# ranker_class.db.drop("predictions")
# relevant_columns = list(set(list(training_data.columns)) - set(ranker_class.factors))
# predictions = ranker_class.sim_processor(training_data)
# ranker_class.db.store("predictions",predictions[predictions["year"]==current_year][relevant_columns])
# ranker_class.db.disconnect()

In [12]:
# predictions[predictions["year"]==current_year][relevant_columns]